<h1 style='text-align: center;'><b>Ensamble Secuencial de Modelos Predictivos</b></h1>
<h3 style='text-align: center;'>Inteligencia Artificial</h3>
<h3 style='text-align: center;'>Grado en Ingeniería Informática - Ingeniería del Software</h3>
<h3 style='text-align: center;'>Universidad de Sevilla</h3>

<h3><b>Implementación de los Algoritmos</b></h3>

En esta sección se proporcionan las diferentes funciones y clases desarrolladas de ensamble secuencial de modelos predictivos.

In [1]:
from sklearn.base import BaseEstimator, clone
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

Se ha establecido una semilla para facilitar la reproducción de los diferentes experimentos.

In [2]:
np.random.seed(357823)

La siguiente función es de gran utilidad para la comprobación de condiciones y será usada a lo largo de toda la implementación de los diferentes algoritmos.

In [3]:
def checkCondition(condition, message=None):
    if (not condition):
        raise Exception(message)

Para facilitar el uso de los diferentes algoritmos de **potenciación del gradiente (Gradient Boosting)** para tareas de regresión, se ha encapsulado toda la funcionalidad en la clase **SequentialRegressor**. Dicha clase necesita de los siguientes parámetros de entrada para su correcto funcionamiento:
- **Estimator:** modelo predictivo que será utilizado para el entrenamiento y ensamblaje secuencial. Dicho modelo debe ser de la biblioteca [Scikit-learn](http://scikit-learn.org) o en su defecto un modelo que debe heredar de la clase [BaseEstimator](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html).
- **N_estimators:** número de modelos predictivos ensamblados secuencialmente.
- **Sample_size:** proporción de ejemplos del conjunto de entrenamiento que se ultizará para entrenar cada modelo.
- **Lr:** parámetro de aprendizaje del meta-modelo.
- **Hyperparameters:** hiperparámetros de los modelos predictivos.

Las funciones que implementa la clase SequentialRegressor para el ensamble secuencial de modelos predictivos son las siguientes:
- **Fit:** realiza el entrenamiento de los modelos por potenciación del gradiente. Recibe los atributos y la variable objetivo que constituyen el conjunto de entrenamiento del modelo. Devuelve la lista de los modelos predictivos entrenados.
- **Predict:** realiza la predicción del meta-modelo para unos atributos dados, a partir de las predicciones de cada uno de los modelos en secuencia entrenados. Devuelve la predicción final del meta-modelo.
- **Evaluate:** realiza una validación cruzada con k-pliegues del meta-modelo entrenado. Recibe los atributos y la variable objetivo que constituyen el conjunto de entrenamiento del modelo y el número de pliegues a utilizar para la validación cruzada. Devuelve el rendimiento medio del meta-modelo.

In [4]:
class SequentialRegressor(BaseEstimator):
    
    def __init__(self, estimator, n_estimators=1,
                sample_size=1., lr=1., hyperparameters=None):
        checkCondition(isinstance(estimator, BaseEstimator), "Estimator must be an instance of Scikit-learn class")
        checkCondition(n_estimators >= 1, "Number of estimators must be bigger or equals to 1")
        checkCondition(0. < sample_size <= 1., "The sample size must be between 0 (excluded) and 1")
        if hyperparameters:
            estimator.set_params(**hyperparameters)
        self.estimator = estimator
        self.n_estimators = n_estimators
        self.sample_size = sample_size
        self.lr = lr
        self.hyperparameters = hyperparameters

    def fit(self, attributes, objective):
        attributes = np.asarray(attributes)
        objective = np.asarray(objective).ravel()
        checkCondition(attributes.size > 0, "The attributes can't be empty")
        checkCondition(objective.size > 0, "The objective can't be empty")

        rows = attributes.shape[0]
        prediction = np.zeros(rows)
        self.models = []

        for i in range(self.n_estimators): # Para cada modelo a entrenar
            remainder_i = objective - prediction # Variable objetivo es el residuo
            n_samples_i = int(self.sample_size * rows)
            selected = np.random.choice(rows, size=n_samples_i, replace=False) # Selección aleatoria del conjunto de entrenamiento
            samples_i = attributes[selected]
            objective_i = remainder_i[selected]
            model = clone(self.estimator) # Aseguramos así que utilizamos siempre un modelo nuevo no entrenado
            model.fit(samples_i, objective_i)
            self.models.append(model)
            prediction_i = model.predict(attributes)
            prediction = prediction + self.lr * prediction_i # Actualizamos la predicción para el residuo de la siguiente iteración
        return self.models

    def predict(self, attributes):
        checkCondition(hasattr(self, 'models') and len(self.models) > 0, "This SequentialRegressor instance is not fitted yet." +
                       "Call 'fit' with appropriate arguments before using this estimator")
        attributes = np.asarray(attributes)
        checkCondition(attributes.size > 0, "The attributes can't be empty")
        
        rows = attributes.shape[0]
        predictions = np.zeros(rows)
        for model in self.models: # Para cada modelo entrenado
            predictions += self.lr * model.predict(attributes) # Suma acumulada de cada predicción con suavizado
        return predictions

    def evaluate(self, attributes, objective, k):
        checkCondition(k >= 1, "The number of partitions must be bigger or equals to 1")
        results = cross_validate(self, attributes, objective, scoring=['r2', 'neg_mean_absolute_error'], cv=k)
        return {'r2': results['test_r2'].mean(), 'neg_mean_absolute_error': results['test_neg_mean_absolute_error'].mean()}

<h2>Experimentación</h2>

En esta sección se proporcionan los diferentes experimentos realizados para la comprobación del correcto funcionamiento de los diferentes algoritmos y la exploración del espacio de hiperparámetros de estos.

<h4>Tratamiento de los Conjuntos de Datos</h4>

Primero extraemos el contenido de los CSVs propuestos y separamos el contenido de estos en los atributos y variable objetivo para su correcto funcionamiento con la clase propuesta. 
A continuación es necesario tratar debidamente estos datos. Para el archivo que contiene la información sobre el precio de viviendas, codificamos las columnas que tienen valores en formato texto a variables númericas que las representen y finalmente normalizamos todas las columnas de entrenamiento (excluimos la variable objetivo), para que todos los valores se encuentren en el rango [0,1].

In [5]:
houses = pd.read_csv("house_prices.csv") 
houses_attributes = houses.iloc[:,:-1] 
houses_objective = houses.iloc[:,-1:] 
houses.head()

,GarageCars,Condition2,YearBuilt,GarageYrBlt,LandContour,LowQualFinSF,HouseStyle,GarageType,MSSubClass,WoodDeckSF,...,MiscVal,BsmtExposure,OpenPorchSF,ExterCond,Fireplaces,FullBath,BsmtQual,MiscFeature,PoolQC,SalePrice
0,2,Norm,1962,1977.0,Lvl,0,1Story,Detchd,20,0,...,0,none,0,TA,0,1,TA,none,none,132000
1,0,Norm,1914,0.0,Lvl,0,2.5Unf,none,75,0,...,0,none,291,TA,1,2,TA,none,none,128000
2,2,Norm,1999,1999.0,Lvl,0,1Story,Attchd,20,0,...,0,Av,35,TA,0,2,Gd,none,none,192000
3,1,Norm,1948,1948.0,Bnk,0,2Story,Attchd,20,103,...,0,none,0,Gd,0,3,TA,none,none,225000
4,2,Norm,1950,1950.0,Lvl,0,1Story,Detchd,20,0,...,0,none,29,TA,0,1,none,none,none,109900


In [6]:
houses_discrete_attributes = houses_attributes.select_dtypes(include=['object', 'string']).columns 
encoder = OrdinalEncoder() 
houses_attributes[houses_discrete_attributes] = encoder.fit_transform(houses_attributes[houses_discrete_attributes])
houses_attributes.head()

,GarageCars,Condition2,YearBuilt,GarageYrBlt,LandContour,LowQualFinSF,HouseStyle,GarageType,MSSubClass,WoodDeckSF,...,SaleType,MiscVal,BsmtExposure,OpenPorchSF,ExterCond,Fireplaces,FullBath,BsmtQual,MiscFeature,PoolQC
0,2,2.0,1962,1977.0,3.0,0,2.0,5.0,20,0,...,7.0,0,3.0,0,3.0,0,1,3.0,4.0,2.0
1,0,2.0,1914,0.0,3.0,0,4.0,6.0,75,0,...,7.0,0,3.0,291,3.0,1,2,3.0,4.0,2.0
2,2,2.0,1999,1999.0,3.0,0,2.0,1.0,20,0,...,7.0,0,0.0,35,3.0,0,2,2.0,4.0,2.0
3,1,2.0,1948,1948.0,0.0,0,5.0,1.0,20,103,...,7.0,0,3.0,0,1.0,0,3,3.0,4.0,2.0
4,2,2.0,1950,1950.0,3.0,0,2.0,5.0,20,0,...,7.0,0,3.0,29,3.0,0,1,4.0,4.0,2.0


In [7]:
normalizer = MinMaxScaler(feature_range=(0, 1)) 
houses_attributes = pd.DataFrame(
    normalizer.fit_transform(houses_attributes), 
    columns=houses_attributes.columns ) 
houses_attributes.head()

,GarageCars,Condition2,YearBuilt,GarageYrBlt,LandContour,LowQualFinSF,HouseStyle,GarageType,MSSubClass,WoodDeckSF,...,SaleType,MiscVal,BsmtExposure,OpenPorchSF,ExterCond,Fireplaces,FullBath,BsmtQual,MiscFeature,PoolQC
0,0.50,0.4,0.649254,0.984072,1.0,0.0,0.285714,0.833333,0.000000,0.000000,...,1.0,0.0,1.0,0.000000,1.000000,0.000000,0.333333,0.75,1.0,1.0
1,0.00,0.4,0.291045,0.000000,1.0,0.0,0.571429,1.000000,0.323529,0.000000,...,1.0,0.0,1.0,0.716749,1.000000,0.333333,0.666667,0.75,1.0,1.0
2,0.50,0.4,0.925373,0.995022,1.0,0.0,0.285714,0.166667,0.000000,0.000000,...,1.0,0.0,0.0,0.086207,1.000000,0.000000,0.666667,0.50,1.0,1.0
3,0.25,0.4,0.544776,0.969637,0.0,0.0,0.714286,0.166667,0.000000,0.162205,...,1.0,0.0,1.0,0.000000,0.333333,0.000000,1.000000,0.75,1.0,1.0
4,0.50,0.4,0.559701,0.970632,1.0,0.0,0.285714,0.833333,0.000000,0.000000,...,1.0,0.0,1.0,0.071429,1.000000,0.000000,0.333333,1.00,1.0,1.0


Para el archivo que contiene la información sobre la enfermedad de Parkinson el tratamiento es el mismo, sin embargo no es necesario el paso de codificación ya que todas las columnas presentan valores númericos.

In [8]:
parkinsons = pd.read_csv("parkinsons.csv")
parkinsons_attributes = parkinsons.iloc[:,:-1]
parkinsons_objective = parkinsons.iloc[:,-1:]
parkinsons.head()

,age,sex,test_time,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,Shimmer,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE,total_UPDRS
0,49,0,194.890,0.01674,0.000156,0.01055,0.00750,0.03166,0.09713,0.833,0.05832,0.04904,0.06635,0.17497,0.194060,12.612,0.76658,0.66783,0.44528,25.011
1,66,1,135.410,0.00520,0.000029,0.00179,0.00220,0.00536,0.05364,0.471,0.03006,0.02792,0.03804,0.09019,0.065038,15.346,0.63731,0.54549,0.25823,32.000
2,63,0,61.338,0.00548,0.000043,0.00259,0.00259,0.00776,0.03919,0.353,0.02147,0.02547,0.03266,0.06440,0.017563,21.109,0.56231,0.64180,0.20707,39.674
3,75,0,181.350,0.00701,0.000055,0.00270,0.00371,0.00810,0.02869,0.269,0.01196,0.01599,0.02768,0.03589,0.017749,20.992,0.50528,0.76762,0.24917,24.925
4,55,0,168.840,0.01601,0.000113,0.00883,0.00775,0.02650,0.04998,0.494,0.02814,0.02878,0.03887,0.08443,0.091321,17.524,0.60864,0.73426,0.47043,26.230


In [9]:
normalizer = MinMaxScaler(feature_range=(0, 1))
parkinsons_attributes = pd.DataFrame(
    normalizer.fit_transform(parkinsons_attributes),
    columns=parkinsons_attributes.columns
)
parkinsons_attributes.head()

,age,sex,test_time,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,Shimmer,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,0.265306,0.0,0.906258,0.160448,0.394159,0.178640,0.102271,0.178736,0.482192,0.492940,0.572932,0.313121,0.232539,0.572961,0.258728,0.324861,0.770456,0.431336,0.595707
1,0.612245,1.0,0.635590,0.044070,0.065745,0.025520,0.025604,0.025517,0.258363,0.270718,0.286871,0.172386,0.128638,0.286919,0.086154,0.405950,0.594343,0.079562,0.331706
2,0.551020,0.0,0.298520,0.046894,0.102573,0.039504,0.031245,0.039499,0.183994,0.198281,0.199919,0.156061,0.108893,0.199906,0.022654,0.576877,0.492166,0.356490,0.259499
3,0.795918,0.0,0.844643,0.062324,0.132917,0.041426,0.047447,0.041480,0.129954,0.146716,0.103654,0.092890,0.090615,0.103715,0.022903,0.573407,0.414471,0.718270,0.318918
4,0.387755,0.0,0.787716,0.153086,0.283677,0.148575,0.105887,0.148675,0.239527,0.284837,0.267436,0.178117,0.131684,0.267485,0.121309,0.470548,0.555285,0.622347,0.631203


<h4>Potenciación del Gradiente con CART</h4>

Una vez tenemos la información almacenada correctamente, pasamos a la experimentación sobre estos conjuntos de datos. Para ello primero utilizaremos el modelo predictivo [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) y realizaremos una **validación por retención**.

In [10]:
(training_attributes_houses, test_attributes_houses,
 training_objetive_houses, test_objective_houses) = train_test_split(houses_attributes, houses_objective, test_size=.2)
(training_attributes_parkinsons, test_attributes_parkinsons,
 training_objetive_parkinsons, test_objective_parkinsons) = train_test_split(parkinsons_attributes, parkinsons_objective, test_size=.2)

In [11]:
hyperparametersTree = {
    'max_depth' : 10,
    'min_samples_split': 2
}
housesTree = SequentialRegressor(DecisionTreeRegressor(), n_estimators=4, sample_size=0.75, lr=0.5, hyperparameters=hyperparametersTree)
parkinsonsTree = SequentialRegressor(DecisionTreeRegressor(), n_estimators=4, sample_size=0.75, lr=0.5, hyperparameters=hyperparametersTree)

In [12]:
housesTree.fit(training_attributes_houses, training_objetive_houses)
housesPredictions = housesTree.predict(test_attributes_houses)
parkinsonsTree.fit(training_attributes_parkinsons, training_objetive_parkinsons)
parkinsonsPredictions = parkinsonsTree.predict(test_attributes_parkinsons)

In [13]:
print("Coeficiente de determinación (Viviendas): " + str(r2_score(test_objective_houses, housesPredictions)))
print("Coeficiente de determinación (Parkinson): " + str(r2_score(test_objective_parkinsons, parkinsonsPredictions)))
print("Error absoluto medio (Viviendas): " + str(mean_absolute_error(test_objective_houses, housesPredictions)))
print("Error absoluto medio (Parkinson): " + str(mean_absolute_error(test_objective_parkinsons, parkinsonsPredictions)))

Coeficiente de determinación (Viviendas): 0.6577758401970466
Coeficiente de determinación (Parkinson): 0.8419887246197574
Error absoluto medio (Viviendas): 29325.109078521036
Error absoluto medio (Parkinson): 3.049775979490048


A continuación realiamos una **validación cruzada por 10-pliegues** con los conjuntos de datos.

In [14]:
housesTree = SequentialRegressor(DecisionTreeRegressor(), n_estimators=4, sample_size=0.75, lr=0.5, hyperparameters=hyperparametersTree)
parkinsonsTree = SequentialRegressor(DecisionTreeRegressor(), n_estimators=4, sample_size=0.75, lr=0.5, hyperparameters=hyperparametersTree)
housesResults = housesTree.evaluate(houses_attributes, houses_objective, 10)
parkinsonsResults = parkinsonsTree.evaluate(parkinsons_attributes, parkinsons_objective, 10)
print("Coeficiente de determinación (Viviendas): " + str(housesResults['r2']))
print("Coeficiente de determinación (Parkinson): " + str(parkinsonsResults['r2']))
print("Error absoluto medio (Viviendas): " + str(-housesResults['neg_mean_absolute_error']))
print("Error absoluto medio (Parkinson): " + str(-parkinsonsResults['neg_mean_absolute_error']))

Coeficiente de determinación (Viviendas): 0.6414133455646611
Coeficiente de determinación (Parkinson): 0.8371597722303535
Error absoluto medio (Viviendas): 29888.189670346463
Error absoluto medio (Parkinson): 3.0107721851961307


<h4>Potenciación del Gradiente con KNN</h4>

Para continuar con la experimentación realizamos las mismas pruebas anteriormente realizadas pero ahora con el modelo predictivo [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html). Comenzamos por la **validación por retención**.

In [15]:
hyperparametersKNN = {
    'n_neighbors' : 5,
    'metric': 'euclidean'
}
housesKNN = SequentialRegressor(KNeighborsRegressor(), n_estimators=4, sample_size=0.75, lr=0.5, hyperparameters=hyperparametersKNN)
parkinsonsKNN = SequentialRegressor(KNeighborsRegressor(), n_estimators=4, sample_size=0.75, lr=0.5, hyperparameters=hyperparametersKNN)

In [16]:
housesKNN.fit(training_attributes_houses, training_objetive_houses)
housesPredictions = housesKNN.predict(test_attributes_houses)
parkinsonsKNN.fit(training_attributes_parkinsons, training_objetive_parkinsons)
parkinsonsPredictions = parkinsonsKNN.predict(test_attributes_parkinsons)

In [17]:
print("Coeficiente de determinación (Viviendas): " + str(r2_score(test_objective_houses, housesPredictions)))
print("Coeficiente de determinación (Parkinson): " + str(r2_score(test_objective_parkinsons, parkinsonsPredictions)))
print("Error absoluto medio (Viviendas): " + str(mean_absolute_error(test_objective_houses, housesPredictions)))
print("Error absoluto medio (Parkinson): " + str(mean_absolute_error(test_objective_parkinsons, parkinsonsPredictions)))

Coeficiente de determinación (Viviendas): 0.6718099617369188
Coeficiente de determinación (Parkinson): 0.5358055854298482
Error absoluto medio (Viviendas): 28768.631728571432
Error absoluto medio (Parkinson): 5.08990815115


A continuación continuamos con la **validación cruzada por 10-pliegues**.

In [18]:
housesKNN = SequentialRegressor(KNeighborsRegressor(), n_estimators=4, sample_size=0.75, lr=0.5, hyperparameters=hyperparametersKNN)
parkinsonsKNN = SequentialRegressor(KNeighborsRegressor(), n_estimators=4, sample_size=0.75, lr=0.5, hyperparameters=hyperparametersKNN)
housesResults = housesKNN.evaluate(houses_attributes, houses_objective, 10)
parkinsonsResults = parkinsonsKNN.evaluate(parkinsons_attributes, parkinsons_objective, 10)
print("Coeficiente de determinación (Viviendas): " + str(housesResults['r2']))
print("Coeficiente de determinación (Parkinson): " + str(parkinsonsResults['r2']))
print("Error absoluto medio (Viviendas): " + str(-housesResults['neg_mean_absolute_error']))
print("Error absoluto medio (Parkinson): " + str(-parkinsonsResults['neg_mean_absolute_error']))

Coeficiente de determinación (Viviendas): 0.6470140269185698
Coeficiente de determinación (Parkinson): 0.45677398150572646
Error absoluto medio (Viviendas): 29389.375615714285
Error absoluto medio (Parkinson): 5.343921980724999


<h4>CART</h4>

Para ver la efectividad del algoritmo de potenciación del gradiente realizaremos los mismos experimentos con un único modelo predictivo de cada tipo. Comenzamos con el algoritmo CART y **validación por retención**.

In [19]:
housesTree = DecisionTreeRegressor(max_depth=10, min_samples_split=2)
parkinsonsTree = DecisionTreeRegressor(max_depth=10, min_samples_split=2)

In [20]:
housesTree.fit(training_attributes_houses, training_objetive_houses)
housesPredictions = housesTree.predict(test_attributes_houses)
parkinsonsTree.fit(training_attributes_parkinsons, training_objetive_parkinsons)
parkinsonsPredictions = parkinsonsTree.predict(test_attributes_parkinsons)

In [21]:
print("Coeficiente de determinación (Viviendas): " + str(r2_score(test_objective_houses, housesPredictions)))
print("Coeficiente de determinación (Parkinson): " + str(r2_score(test_objective_parkinsons, parkinsonsPredictions)))
print("Error absoluto medio (Viviendas): " + str(mean_absolute_error(test_objective_houses, housesPredictions)))
print("Error absoluto medio (Parkinson): " + str(mean_absolute_error(test_objective_parkinsons, parkinsonsPredictions)))

Coeficiente de determinación (Viviendas): 0.6111689915147074
Coeficiente de determinación (Parkinson): 0.881663193370734
Error absoluto medio (Viviendas): 34126.17865709439
Error absoluto medio (Parkinson): 1.4747014369738067


A continuación se realiza el mismo experimento con **validación cruzada por 10-pliegues**.

In [22]:
housesTree = DecisionTreeRegressor(max_depth=10, min_samples_split=2)
parkinsonsTree = DecisionTreeRegressor(max_depth=10, min_samples_split=2)
housesResults = cross_validate(housesTree, houses_attributes, houses_objective, scoring=['r2', 'neg_mean_absolute_error'], cv=10)
parkinsonsResults = cross_validate(parkinsonsTree, parkinsons_attributes, parkinsons_objective, scoring=['r2', 'neg_mean_absolute_error'], cv=10)
print("Coeficiente de determinación (Viviendas): " + str(housesResults['test_r2'].mean()))
print("Coeficiente de determinación (Parkinson): " + str(parkinsonsResults['test_r2'].mean()))
print("Error absoluto medio (Viviendas): " + str(-housesResults['test_neg_mean_absolute_error'].mean()))
print("Error absoluto medio (Parkinson): " + str(-parkinsonsResults['test_neg_mean_absolute_error'].mean()))

Coeficiente de determinación (Viviendas): 0.47064096701735236
Coeficiente de determinación (Parkinson): 0.8515609328940126
Error absoluto medio (Viviendas): 33203.51605587966
Error absoluto medio (Parkinson): 1.8189509033608515


<h4>KNN</h4>

Continuamos con el algoritmo KNN y **validación por retención**.

In [23]:
housesKNN = KNeighborsRegressor(n_neighbors=5, metric='euclidean')
parkinsonsKNN = KNeighborsRegressor(n_neighbors=5, metric='euclidean')

In [24]:
housesKNN.fit(training_attributes_houses, training_objetive_houses)
housesPredictions = housesKNN.predict(test_attributes_houses)
parkinsonsKNN.fit(training_attributes_parkinsons, training_objetive_parkinsons)
parkinsonsPredictions = parkinsonsKNN.predict(test_attributes_parkinsons)

In [25]:
print("Coeficiente de determinación (Viviendas): " + str(r2_score(test_objective_houses, housesPredictions)))
print("Coeficiente de determinación (Parkinson): " + str(r2_score(test_objective_parkinsons, parkinsonsPredictions)))
print("Error absoluto medio (Viviendas): " + str(mean_absolute_error(test_objective_houses, housesPredictions)))
print("Error absoluto medio (Parkinson): " + str(mean_absolute_error(test_objective_parkinsons, parkinsonsPredictions)))

Coeficiente de determinación (Viviendas): 0.6936083453911583
Coeficiente de determinación (Parkinson): 0.5741096990316554
Error absoluto medio (Viviendas): 29221.305357142857
Error absoluto medio (Parkinson): 4.6179443000000004


Finalmente realizamos la **validación cruzada por 10-pliegues**.

In [26]:
housesKNN = KNeighborsRegressor(n_neighbors=5, metric='euclidean')
parkinsonsKNN = KNeighborsRegressor(n_neighbors=5, metric='euclidean')
housesResults = cross_validate(housesKNN, houses_attributes, houses_objective, scoring=['r2', 'neg_mean_absolute_error'], cv=10)
parkinsonsResults = cross_validate(parkinsonsKNN, parkinsons_attributes, parkinsons_objective, scoring=['r2', 'neg_mean_absolute_error'], cv=10)
print("Coeficiente de determinación (Viviendas): " + str(housesResults['test_r2'].mean()))
print("Coeficiente de determinación (Parkinson): " + str(parkinsonsResults['test_r2'].mean()))
print("Error absoluto medio (Viviendas): " + str(-housesResults['test_neg_mean_absolute_error'].mean()))
print("Error absoluto medio (Parkinson): " + str(-parkinsonsResults['test_neg_mean_absolute_error'].mean()))

Coeficiente de determinación (Viviendas): 0.6736937651211619
Coeficiente de determinación (Parkinson): 0.5326926157166312
Error absoluto medio (Viviendas): 28452.66535714286
Error absoluto medio (Parkinson): 4.718700940000001


<h3>Experimentación con hiperparámetros</h3>

A continuación procedemos a experimentar con diferentes hiperparámetros mediante una **busqueda por rejilla** para analizar cual es el mejor conjunto de ellos. Para ello utilizaremos una función, para facilitar la experimentación, que toma como parámetros los rangos de los hiperparámetros a analizar mediante la busqueda en rejilla de [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [27]:
def busqueda_hiperparametros(estimator,hnames_estimator,hvalues_estimator,n_estimators,sample_size,lr,kpliegues,atributos_entrenamiento,objetivo_entrenamiento):
    tree = SequentialRegressor(estimator)
    rejilla_de_hiperparametros = {
    'n_estimators':n_estimators,    
    'sample_size': sample_size,
    'lr':lr   
    }
    for i in range(len(hnames_estimator)):#toma los nombres y los valores de los hiperparámetros del estimador correspondiente y lo añade al diccionario de hiperparámetros.
        name = 'estimator__'+ hnames_estimator[i]
        value = hvalues_estimator[i]
        rejilla_de_hiperparametros[name] = value
    busqueda_en_rejilla = GridSearchCV(tree,rejilla_de_hiperparametros,scoring = 'r2',cv=kpliegues)
    busqueda_en_rejilla.fit(atributos_entrenamiento,objetivo_entrenamiento)
    return [busqueda_en_rejilla.best_params_,busqueda_en_rejilla.best_score_] 

<h4>Evaluación del modelo de ensamble secuencial</h4>

Procedemos a analizar los hiperparámetros más útiles para el estimador **CART** con validación cruzada

In [28]:
estimator = DecisionTreeRegressor()
hnames_estimator = ['max_depth','min_samples_split']
hvalues_estimator = [[3,5,7,10],[2,5,10]]
n_estimators = range(4,10)
sample_size = [i/10 for i in range(5,11,2)]
lr = [i/10 for i in range(5,11,2)]
kpliegues = 10
Hiperparametros_Houses = busqueda_hiperparametros(estimator,hnames_estimator,hvalues_estimator,n_estimators,sample_size,lr,kpliegues,houses_attributes,houses_objective)
Hiperparametros_Parkinson = busqueda_hiperparametros(estimator,hnames_estimator,hvalues_estimator,n_estimators,sample_size,lr,kpliegues,parkinsons_attributes,parkinsons_objective)
print("Conjunto de hiperparámetros(Viviendas):" )
print(Hiperparametros_Houses[0])
print("Coeficiente de determinación para el mejor conjunto de hiperparámetros = "+ str(Hiperparametros_Houses[1]))
print("Conjunto de hiperparámetros(Parkinson):")
print(Hiperparametros_Parkinson[0])
print("Coeficiente de determinación para el mejor conjunto de hiperparámetros = "+ str(Hiperparametros_Parkinson[1]))

KeyboardInterrupt: 

Ahora continuamos analizando hiperparámetros pero con el estimador **KNN** con validación cruzada

In [34]:
estimator = KNeighborsRegressor()
hnames_estimator = ['n_neighbors','weights','p']
hvalues_estimator = [[3,5,7],['uniform', 'distance'],[1, 2]]# p1 = Manhattan, p2 = Euclidean
n_estimators = range(4,10)
sample_size = [i/10 for i in range(5,11,2)]
lr = [i/10 for i in range(5,11,2)]
kpliegues = 10
Hiperparametros_Houses = busqueda_hiperparametros(estimator,hnames_estimator,hvalues_estimator,n_estimators,sample_size,lr,kpliegues,houses_attributes,houses_objective)
Hiperparametros_Parkinson = busqueda_hiperparametros(estimator,hnames_estimator,hvalues_estimator,n_estimators,sample_size,lr,kpliegues,parkinsons_attributes,parkinsons_objective)
print("Conjunto de hiperparámetros(Viviendas):" )
print(Hiperparametros_Houses[0])
print("Coeficiente de determinación para el mejor conjunto de hiperparámetros = "+ str(Hiperparametros_Houses[1]))
print("Conjunto de hiperparámetros(Parkinson):")
print(Hiperparametros_Parkinson[0])
print("Coeficiente de determinación para el mejor conjunto de hiperparámetros = "+ str(Hiperparametros_Parkinson[1]))

Conjunto de hiperparámetros(Viviendas):
{'estimator__n_neighbors': 7, 'estimator__p': 1, 'estimator__weights': 'distance', 'lr': 0.7, 'n_estimators': 9, 'sample_size': 0.5}
Coeficiente de determinación para el mejor conjunto de hiperparámetros = 0.7402516940036891
Conjunto de hiperparámetros(Parkinson):
{'estimator__n_neighbors': 5, 'estimator__p': 1, 'estimator__weights': 'distance', 'lr': 0.5, 'n_estimators': 9, 'sample_size': 0.9}
Coeficiente de determinación para el mejor conjunto de hiperparámetros = 0.611870910121626


Hay que tener en cuenta que para analizar un rango de hiperparámetros más alto, el tiempo de cálculo de busqueda en rejilla será mayor.

Una vez conocidos los mejores valores de los hiperparámetros de cada modelo en ensamble secuencial, procedemos a calcular el **coeficiente de determinación y el error absoluto medio**.

**Algoritmo CART con mejor conjunto de hiperparámetros**

In [41]:
hyperparametersTreeHouses = {
    'max_depth' : 5,
    'min_samples_split': 10
}
hyperparametersTreeParkinson = {
    'max_depth' : 10,
    'min_samples_split': 5
}

In [42]:
housesTree = SequentialRegressor(DecisionTreeRegressor(), n_estimators=8, sample_size=0.9, lr=0.5, hyperparameters=hyperparametersTreeHouses)
parkinsonsTree = SequentialRegressor(DecisionTreeRegressor(), n_estimators=8, sample_size=0.9, lr=0.5, hyperparameters=hyperparametersTreeParkinson)
housesResults = housesTree.evaluate(houses_attributes, houses_objective, 10)
parkinsonsResults = parkinsonsTree.evaluate(parkinsons_attributes, parkinsons_objective, 10)
print("Coeficiente de determinación (Viviendas): " + str(housesResults['r2']))
print("Coeficiente de determinación (Parkinson): " + str(parkinsonsResults['r2']))
print("Error absoluto medio (Viviendas): " + str(-housesResults['neg_mean_absolute_error']))
print("Error absoluto medio (Parkinson): " + str(-parkinsonsResults['neg_mean_absolute_error']))

Coeficiente de determinación (Viviendas): 0.7034886782895259
Coeficiente de determinación (Parkinson): 0.9152808633950024
Error absoluto medio (Viviendas): 26269.38651411226
Error absoluto medio (Parkinson): 1.7322292925307603


**Algoritmo KNN con mejor conjunto de hiperparámetros**

In [37]:
hyperparametersKNNHouses = {
    'n_neighbors' : 7,
    'metric': 'manhattan',
    'weights': 'distance'
}
hyperparametersKNNParkinson = {
    'n_neighbors' : 5,
    'metric': 'manhattan',
    'weights': 'distance'
}

In [38]:
housesKNN = SequentialRegressor(KNeighborsRegressor(), n_estimators=9, sample_size=0.5, lr=0.7, hyperparameters=hyperparametersKNNHouses)
parkinsonsKNN = SequentialRegressor(KNeighborsRegressor(), n_estimators=9, sample_size=0.9, lr=0.5, hyperparameters=hyperparametersKNNParkinson)
housesResults = housesKNN.evaluate(houses_attributes, houses_objective, 10)
parkinsonsResults = parkinsonsKNN.evaluate(parkinsons_attributes, parkinsons_objective, 10)
print("Coeficiente de determinación (Viviendas): " + str(housesResults['r2']))
print("Coeficiente de determinación (Parkinson): " + str(parkinsonsResults['r2']))
print("Error absoluto medio (Viviendas): " + str(-housesResults['neg_mean_absolute_error']))
print("Error absoluto medio (Parkinson): " + str(-parkinsonsResults['neg_mean_absolute_error']))

Coeficiente de determinación (Viviendas): 0.7439238963343462
Coeficiente de determinación (Parkinson): 0.6096091613818606
Error absoluto medio (Viviendas): 24567.568054759802
Error absoluto medio (Parkinson): 4.279480915539421


<h4>Evaluación de modelos individuales con los mejores hiperparámetros</h4>

Usaremos la misma función definida anteriormente pero para usar un modelo predictivo individualmente y no la clase de ensamblado secuencial.

In [49]:
def busqueda_hiperparametrosU(estimator,hnames_estimator,hvalues_estimator,n_estimators,sample_size,lr,kpliegues,atributos_entrenamiento,objetivo_entrenamiento):
    tree = estimator
    rejilla_de_hiperparametros = {}
    for i in range(len(hnames_estimator)):#toma los nombres y los valores de los hiperparámetros del estimador correspondiente y lo añade al diccionario de hiperparámetros.
        name =  hnames_estimator[i]
        value = hvalues_estimator[i]
        rejilla_de_hiperparametros[name] = value
    busqueda_en_rejilla = GridSearchCV(tree,rejilla_de_hiperparametros,scoring = 'r2',cv=kpliegues)
    busqueda_en_rejilla.fit(atributos_entrenamiento,objetivo_entrenamiento)
    return [busqueda_en_rejilla.best_params_,busqueda_en_rejilla.best_score_] 

**Modelo KNN**

In [50]:
estimator = KNeighborsRegressor()
hnames_estimator = ['n_neighbors','weights','p']
hvalues_estimator = [[3,5,7],['uniform', 'distance'],[1, 2]]# p1 = Manhattan, p2 = Euclidean
n_estimators = range(4,10)
sample_size = [i/10 for i in range(5,11,2)]
lr = [i/10 for i in range(5,11,2)]
kpliegues = 10
Hiperparametros_Houses = busqueda_hiperparametrosU(estimator,hnames_estimator,hvalues_estimator,n_estimators,sample_size,lr,kpliegues,houses_attributes,houses_objective)
Hiperparametros_Parkinson = busqueda_hiperparametrosU(estimator,hnames_estimator,hvalues_estimator,n_estimators,sample_size,lr,kpliegues,parkinsons_attributes,parkinsons_objective)
print("Conjunto de hiperparámetros(Viviendas):" )
print(Hiperparametros_Houses[0])
print("Coeficiente de determinación para el mejor conjunto de hiperparámetros = "+ str(Hiperparametros_Houses[1]))
print("Conjunto de hiperparámetros(Parkinson):")
print(Hiperparametros_Parkinson[0])
print("Coeficiente de determinación para el mejor conjunto de hiperparámetros = "+ str(Hiperparametros_Parkinson[1]))

Conjunto de hiperparámetros(Viviendas):
{'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
Coeficiente de determinación para el mejor conjunto de hiperparámetros = 0.7131753308742153
Conjunto de hiperparámetros(Parkinson):
{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
Coeficiente de determinación para el mejor conjunto de hiperparámetros = 0.6047311312834615


**Modelo CART**

In [51]:
estimator = DecisionTreeRegressor()
hnames_estimator = ['max_depth','min_samples_split']
hvalues_estimator = [[3,5,7,10],[2,5,10]]
n_estimators = range(4,10)
sample_size = [i/10 for i in range(5,11,2)]
lr = [i/10 for i in range(5,11,2)]
kpliegues = 10
Hiperparametros_Houses = busqueda_hiperparametrosU(estimator,hnames_estimator,hvalues_estimator,n_estimators,sample_size,lr,kpliegues,houses_attributes,houses_objective)
Hiperparametros_Parkinson = busqueda_hiperparametrosU(estimator,hnames_estimator,hvalues_estimator,n_estimators,sample_size,lr,kpliegues,parkinsons_attributes,parkinsons_objective)
print("Conjunto de hiperparámetros(Viviendas):" )
print(Hiperparametros_Houses[0])
print("Coeficiente de determinación para el mejor conjunto de hiperparámetros = "+ str(Hiperparametros_Houses[1]))
print("Conjunto de hiperparámetros(Parkinson):")
print(Hiperparametros_Parkinson[0])
print("Coeficiente de determinación para el mejor conjunto de hiperparámetros = "+ str(Hiperparametros_Parkinson[1]))

Conjunto de hiperparámetros(Viviendas):
{'max_depth': 7, 'min_samples_split': 2}
Coeficiente de determinación para el mejor conjunto de hiperparámetros = 0.5591763133192977
Conjunto de hiperparámetros(Parkinson):
{'max_depth': 10, 'min_samples_split': 2}
Coeficiente de determinación para el mejor conjunto de hiperparámetros = 0.8469278108639811


Procedemos a calcular el coeficiente de determinación y el error absoluto medio de los modelos únicos con los hiperparámetros.

**KNN** con los mejores hiperparámetros

In [53]:
housesKNN = KNeighborsRegressor(n_neighbors=7, metric='manhattan',weights ='distance')
parkinsonsKNN = KNeighborsRegressor(n_neighbors=5, metric='manhattan',weights ='distance')
housesResults = cross_validate(housesKNN, houses_attributes, houses_objective, scoring=['r2', 'neg_mean_absolute_error'], cv=10)
parkinsonsResults = cross_validate(parkinsonsKNN, parkinsons_attributes, parkinsons_objective, scoring=['r2', 'neg_mean_absolute_error'], cv=10)
print("Coeficiente de determinación (Viviendas): " + str(housesResults['test_r2'].mean()))
print("Coeficiente de determinación (Parkinson): " + str(parkinsonsResults['test_r2'].mean()))
print("Error absoluto medio (Viviendas): " + str(-housesResults['test_neg_mean_absolute_error'].mean()))
print("Error absoluto medio (Parkinson): " + str(-parkinsonsResults['test_neg_mean_absolute_error'].mean()))

Coeficiente de determinación (Viviendas): 0.7131753308742153
Coeficiente de determinación (Parkinson): 0.6047311312834615
Error absoluto medio (Viviendas): 25591.467621021624
Error absoluto medio (Parkinson): 4.2671918508687545


**CART** con los mejores hiperparámetros

In [54]:
housesTree = DecisionTreeRegressor(max_depth=7, min_samples_split=2)
parkinsonsTree = DecisionTreeRegressor(max_depth=10, min_samples_split=2)
housesResults = cross_validate(housesTree, houses_attributes, houses_objective, scoring=['r2', 'neg_mean_absolute_error'], cv=10)
parkinsonsResults = cross_validate(parkinsonsTree, parkinsons_attributes, parkinsons_objective, scoring=['r2', 'neg_mean_absolute_error'], cv=10)
print("Coeficiente de determinación (Viviendas): " + str(housesResults['test_r2'].mean()))
print("Coeficiente de determinación (Parkinson): " + str(parkinsonsResults['test_r2'].mean()))
print("Error absoluto medio (Viviendas): " + str(-housesResults['test_neg_mean_absolute_error'].mean()))
print("Error absoluto medio (Parkinson): " + str(-parkinsonsResults['test_neg_mean_absolute_error'].mean()))

Coeficiente de determinación (Viviendas): 0.40127009885222176
Coeficiente de determinación (Parkinson): 0.8447551294661606
Error absoluto medio (Viviendas): 32708.005599707238
Error absoluto medio (Parkinson): 1.8463439532080568
